<h1 style="border: thick double #32a1ce; text-align:center;border-radius:35px">
Modélisation statistiques
</h1>

In [24]:
# Librairies standards
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline



import warnings
warnings.filterwarnings('ignore')
from pathlib import Path


In [8]:
# Définition du répertoire de travail
HOME_DIR = Path.cwd().parent
DATA_DIR = Path(HOME_DIR, "notebook")
print(f"Work directory: {HOME_DIR} \nData directory: {DATA_DIR}")

Work directory: /home/onyxia/work/projet-Python/projet 
Data directory: /home/onyxia/work/projet-Python/projet/notebook


In [10]:
Principal_data4 = pd.read_csv(Path(DATA_DIR, "Principal_data4.csv"))

<h1 style="border: thick double #32a1ce; text-align:center;border-radius:35px">
I- feature scaling des variables
</h1>

Les variables de notre jeu de données ayant une très grande échelle, nous allons faire du feature scaling (les remettre à l'echelle) pour rendre nos données homogènes. La méthode privilégiée dans notre cas est la standardisation car les méthodes (regression logistique ) utilisées reposent sur l'hypothèse de normalité.

In [11]:
Principal_data4.columns

Index(['cla_age_5', 'sexe', 'dept', 'prev', 'Niveau prioritaire', 'no2', 'o3',
       'pm10'],
      dtype='object')

In [12]:
col_num = Principal_data4.select_dtypes(['int64','float64']).columns
col_num

Index(['sexe', 'dept', 'prev', 'no2', 'o3', 'pm10'], dtype='object')

In [13]:
Var_cat=Principal_data4.select_dtypes(['object']).columns
Var_cat

Index(['cla_age_5', 'Niveau prioritaire'], dtype='object')

In [14]:
var_norm=[ 'sexe', 'dept', 'prev', 'no2', 'o3', 'pm10']
var_cat=['cla_age_5', 'Niveau prioritaire']

In [15]:
Principal_data4[var_norm].isnull().sum()

sexe    0
dept    0
prev    0
no2     0
o3      0
pm10    0
dtype: int64

In [16]:
Principal_data4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cla_age_5           1352 non-null   object 
 1   sexe                1352 non-null   int64  
 2   dept                1352 non-null   int64  
 3   prev                1352 non-null   float64
 4   Niveau prioritaire  1352 non-null   object 
 5   no2                 1352 non-null   float64
 6   o3                  1352 non-null   float64
 7   pm10                1352 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 84.6+ KB


<h2 style= "text-align:left">
I.1- Standardisation  des variables numériques 
</h2>

<h3 style= "text-align:left">
I.1.a-  Standarscaler 
</h3>

In [20]:
# standardisation des variables numériques
Principal_data4[var_norm] = StandardScaler().fit_transform(Principal_data4[var_norm])

Verifiaction de la normalisation

In [21]:
def verification_normalisation(data_mod):
      for variable_norm in data_mod :
        data_mod_col=data_mod[variable_norm]
        moyenne=np.mean(data_mod_col)
        ecartType = np.std(data_mod_col)

        print("les statistiques pour la variable {}" .format(variable_norm))
        print("La moyenne est de : {} ".format(round(abs(moyenne), 2)))
        print("L'écart type est de : {} ".format(round(abs(ecartType), 2)))
        print(" ")
        print(" ")

In [22]:
verification_normalisation(Principal_data4[var_norm])

les statistiques pour la variable sexe
La moyenne est de : 0.0 
L'écart type est de : 1.0 
 
 
les statistiques pour la variable dept
La moyenne est de : 0.0 
L'écart type est de : 1.0 
 
 
les statistiques pour la variable prev
La moyenne est de : 0.0 
L'écart type est de : 1.0 
 
 
les statistiques pour la variable no2
La moyenne est de : 0.0 
L'écart type est de : 1.0 
 
 
les statistiques pour la variable o3
La moyenne est de : 0.0 
L'écart type est de : 1.0 
 
 
les statistiques pour la variable pm10
La moyenne est de : 0.0 
L'écart type est de : 1.0 
 
 


<h2 style= "text-align:left">
I.2- Encodage des variables catégorielles 
</h2>

In [25]:
# Encodage binaire sexe
Principal_data4["sexe"] = Principal_data4["sexe"].map({1: 1, 2: 0})


In [26]:
# Colonnes à encoder en One-Hot
onehot_features = ["dept", "cla_age_5"]

In [27]:
# Colonnes à encoder en One-Hot
onehot_features = ["dept", "cla_age_5"]

In [30]:
# Colonnes à laisser en l'état (numériques ou déjà transformées)
# Ex: "prev", "no2", "pm10", "o3", "sexe", "Niveau prioritaire", etc.
# Remainder='passthrough' va les conserver sans transformation
# Définition du ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("ohe", OneHotEncoder(drop="first", sparse_output = False) , onehot_features)
    ],
    remainder="passthrough"
)

In [31]:
# 3) Application du ColumnTransformer (ou dans un Pipeline)

df_array = preprocessor.fit_transform(Principal_data4)

# Récupération des noms des colonnes One-Hot
onehot_colnames = preprocessor.named_transformers_["ohe"].get_feature_names_out(onehot_features)
# Récupération des noms de colonnes "passthrough" dans l'ordre
passthrough_cols = [col for col in Principal_data4.columns if col not in onehot_features]

In [32]:
# Fusion des noms dans le bon ordre
final_col_names = list(onehot_colnames) + passthrough_cols

# Création d'un nouveau DataFrame final
df_final = pd.DataFrame(df_array, columns=final_col_names)

df_final.head(2)

,dept_-1.2133166377093498,dept_-1.089194051890415,dept_0.524399563755735,dept_0.6485221495746697,dept_0.7726447353936043,dept_0.896767321212539,dept_1.0208899070314736,cla_age_5_05-09,cla_age_5_10-14,cla_age_5_15-19,...,cla_age_5_85-89,cla_age_5_90-94,cla_age_5_95et+,cla_age_5_tsage,sexe,prev,Niveau prioritaire,no2,o3,pm10
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,NaN,2.660927,"1,2,3",-0.649703,1.103346,0.003698
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,NaN,2.303524,"1,2,3",-1.130356,0.706744,-1.623907


<h1 style= "text-align:left">
Recherche du meilleur algorithme 
</h1>